In [21]:
import os
import csv
import cv2
from PIL import Image
from datetime import datetime, timedelta

In [22]:
key_dir = '402_2key/'
key_files = [os.path.join(key_dir, file) for file in os.listdir(key_dir) if file.endswith('csv')]

In [23]:
key_files

['402_2key/SEN3_00066667BB2C_25590806_120000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590806_070000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590804_190000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590805_020000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590807_090000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590807_140000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590807_010000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590809_230000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590805_100000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590805_050000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590809_130001.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590807_060000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590807_130000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590806_080000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590804_160000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590807_120000.sendat.csv',
 '402_2key/SEN3_00066667BB2C_25590807_070000.sendat.csv',
 '402_2key/SEN

In [24]:
test_key = key_files[0]

In [25]:
with open(test_key, 'r') as csv_file:
    reader = csv.reader(csv_file)
    csv_data = list(reader)

In [26]:
video_dir = '/Volumes/MIMAMORI_101/FFOutput/video'
video_files = [os.path.join(video_dir, file) for file in os.listdir(video_dir) if file.endswith('mp4')]
cap = cv2.VideoCapture(video_dir + video_files[0])

In [58]:
def getFrame(myFrameNumber = 0):
    '''return cv2 frame at frame number'''
    totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if myFrameNumber >= 0 & myFrameNumber <= totalFrames:
        cap.set(cv2.CAP_PROP_POS_FRAMES,myFrameNumber)
    print(totalFrames)
    return totalFrames

# Remove duplicate tag

In [28]:
def clean_data(csv_data):
    temp = csv_data.copy()
    temp.sort(key = lambda x: int(x[0]))
    for i in range(len(temp)):
        try:
            while temp[i][1] == temp[i+1][1]:
                if i+1 == len(temp) -1: 
                    break
                del(temp[i+1])
        except Exception as e:
            pass
    return temp

In [29]:
video_file = video_files[0]

# Parse video date

In [30]:
def parse_date(filePath):
    '''parse video date from filename'''
    filePath = os.path.basename(filePath)
    if filePath.endswith('csv'):
        date_str = filePath[18:33]
        csv_date = datetime.strptime(date_str, '%Y%m%d_%H%M%S')
        csv_date  = csv_date.replace(year=csv_date.year-543)
        return csv_date
    else:
        date_str = filePath[4:-4]
        video_date = datetime.strptime(date_str, '%Y%m%d_%H-%M-%S')
        return video_date
        

In [31]:
def get_delta_frame(csv_date, video_date):
    delta = csv_date - video_date
    delta_frame = delta.total_seconds() * 30
    return delta_frame

In [78]:
def isBetween(csv_date, videofile):
    start = parse_date(videofile)
    if csv_date < start:
        return False
    
    cap = cv2.VideoCapture(video_file)
    cap.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
    duration = cap.get(cv2.CAP_PROP_POS_MSEC) * 1000
    end = start + timedelta(seconds=duration)
    if csv_date in (start , end):
        return True
    else:
        return False

# Get Video

In [87]:
key_file = key_files[0]
video_file = '/Volumes/MIMAMORI_101/FFOutput/video/402-20160806_12-26-33.mp4'
csv_date = parse_date(key_file)

cap = cv2.VideoCapture(video_file)
cap.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
duration = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
start = parse_date(video_file)
end = start + timedelta(seconds=duration)

print(csv_date)
print(start,end,duration)
csv_date in (start,end)

2016-08-06 12:00:00
2016-08-06 12:26:33 2016-08-06 12:53:17.533333 1604.5333333333333


False

In [67]:
for key_file in key_files:
    with open(key_file, 'r') as csv_file:
        reader = csv.reader(csv_file)
        csv_data = list(reader)
    temp = csv_data.copy()
    temp = clean_data(temp)
    csv_date = parse_date(key_file)
    for i, tag in enumerate(temp):
        if int(tag[1]) == 3 and i != len(temp)-1:
            #get Video
            csv_date_new = csv_date + timedelta(seconds=int(tag[0])/30)
            for video_file in video_files:
                print(video_file)
                if isBetween(csv_date_new, video_file):
                    break
            #get frame
            frame = getFrame(get_delta_frame(csv_date, parse_date(video_file)) + int(tag[0]))       
            new_tag = input('Enter new tag: ')
            temp[i][1] = new_tag
            if i != 0 and len(temp[i-1][1]) == 2:
                if temp[i-1][1][0] == new_tag:
                    temp[i-1][1] = new_tag    
                else:
                    temp[i-1][1] = temp[i-1][1][0] + new_tag
            if i != len(temp)-1 and len(temp[i+1][1]) == 2:
                if temp[i+1][1][1] == new_tag:
                    temp[i+1][1] = new_tag    
                else:
                    temp[i+1][1] = new_tag + temp[i+1][1][1]
    temp = clean_data(temp)
    print(temp)

/Volumes/MIMAMORI_101/FFOutput/video/402-20160727_20-35-38.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160728_09-50-10.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160728_09-50-19.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160728_09-52-07.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160728_11-30-49.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160728_11-31-32.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160728_11-39-04.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160728_17-09-25.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160729_12-02-15.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160729_12-06-16.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160729_15-08-26.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160730_12-49-12.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160730_21-01-25.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160731_06-29-29.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160731_17-21-41.mp4
/Volumes/MIMAMORI_101/FFOutput/video/402-20160731_17-33

/Volumes/MIMAMORI_101/FFOutput/video/404-20160729_12-02-59.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160729_12-06-20.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160729_15-08-47.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160730_12-49-51.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160730_21-04-26.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160731_06-31-39.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160731_17-37-36.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160731_17-43-03.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160801_11-46-44.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160801_22-44-10.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160802_12-07-25.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160802_13-19-43.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160802_14-27-45.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160802_15-35-59.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160802_16-31-05.mp4
/Volumes/MIMAMORI_101/FFOutput/video/404-20160802_17-55

/Volumes/MIMAMORI_101/FFOutput/video/406-20160730_21-08-05.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160731_06-34-28.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160731_17-39-43.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160801_11-54-26.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160801_14-46-23.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160801_22-47-13.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160802_12-14-40.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160802_13-19-51.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160802_14-28-08.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160802_15-36-12.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160802_16-31-33.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160802_17-56-50.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160802_20-39-34.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160802_21-52-15.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160802_23-32-10.mp4
/Volumes/MIMAMORI_101/FFOutput/video/406-20160803_00-09

/Volumes/MIMAMORI_101/FFOutput/video/408-20160729_12-04-06.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160729_12-06-29.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160729_15-09-56.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160730_12-50-45.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160730_21-09-58.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160731_06-36-20.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160731_17-42-40.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160801_11-58-22.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160801_14-45-26.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160801_22-49-03.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160802_12-18-00.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160802_13-17-15.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160802_13-20-04.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160802_14-28-39.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160802_15-36-27.mp4
/Volumes/MIMAMORI_101/FFOutput/video/408-20160802_16-32

/Volumes/MIMAMORI_101/FFOutput/video/410-20160728_13-16-39.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160728_17-10-51.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160729_12-04-29.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160729_12-06-33.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160729_15-10-09.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160730_12-51-07.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160730_21-11-49.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160731_06-38-20.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160731_17-45-27.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160801_12-02-09.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160801_14-47-46.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160801_22-50-50.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160802_12-21-16.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160802_13-20-13.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160802_14-29-23.mp4
/Volumes/MIMAMORI_101/FFOutput/video/410-20160802_15-36

TypeError: unsupported operand type(s) for &: 'int' and 'float'

In [65]:
csv_date_new

datetime.datetime(2016, 8, 6, 12, 5, 51, 766667)

In [66]:
video_file

'/Volumes/MIMAMORI_101/FFOutput/video/410-20160902_21-13-52.mp4'

In [63]:
isBetween(csv_date_new, video_file)

False